In [1]:
# Change directory to level up to import functions
import os
os.chdir('..')

In [2]:
import pandas as pd
import glob


In [64]:
def load_most_recent_file():
    # Define the pattern to match timestamped CSV files in the current directory
    pattern = 'data/*_team_sample_data.csv'
    
    # List all files matching the pattern
    files = glob.glob(pattern)
    
    if not files:
        raise FileNotFoundError("No files found matching the pattern.")
    
    # Sort files alphabetically (most recent file will be the last in the sorted list)
    files.sort()
    
    # Get the most recent file
    most_recent_file = files[-1]
    
    # Load the most recent file into a DataFrame
    df = pd.read_csv(most_recent_file)
    
    # Remove the prefix 'data/' or 'data\' and the suffix '_team_sample_data.csv' from the file name
    base_name = most_recent_file.replace('data/', '').replace('data\\', '').replace('_team_sample_data.csv', '')
    
    return df, base_name

In [47]:
def add_proportion_column(df, column_name):
    """
    Adds a new column to the DataFrame with the proportion of each value in the specified column.
    
    Parameters:
    - df: pandas DataFrame
    - column_name: str, the name of the column to calculate proportions for
    
    Returns:
    - pandas DataFrame with an additional column for the proportion of each value
    """
    # Calculate the proportion of each unique value
    value_counts = df[column_name].value_counts(normalize=True)
    
    # Create the new column name
    prop_column_name = f"{column_name}_prop"
    
    # Map these proportions to a new column in the DataFrame
    df[prop_column_name] = df[column_name].map(value_counts)
    
    return df

def calculate_percentiles(df, columns_to_percentile):
    
    for column, order in columns_to_percentile.items():
        if order == 'asc':
            # Create percentile bins using qcut
            df[f'{column}_percentile'] = pd.qcut(df[column], q=100, labels=False, duplicates='drop') + 1
        elif order == 'desc':
            # Create percentile bins in descending order
            df[f'{column}_percentile'] = pd.qcut(-df[column], q=100, labels=False, duplicates='drop') + 1
        else:
            raise ValueError(f"Invalid order '{order}' specified for column '{column}'. Must be 'asc' or 'desc'.")
    
    return df

def calculate_percentiles_partitioned(df, columns_to_percentile_partitioned):
    
    for details in columns_to_percentile_partitioned:
        column_name = details['column_name']
        partition_column = details['partition_column']
        order = details.get('order', 'asc')

        # Validate inputs
        if not column_name or not partition_column:
            raise ValueError("Both 'column_name' and 'partition_column' must be provided.")
        if order not in ['asc', 'desc']:
            raise ValueError("Order must be 'asc' or 'desc'")

        # Apply the computation excluding grouping columns
        if order == 'asc':
            df[f'{column_name}_partitioned_percentile'] = (
                df.groupby(partition_column, group_keys=False)[column_name]
                  .apply(lambda x: pd.qcut(x, q=100, labels=False, duplicates='drop') + 1)
                  .reset_index(level=0, drop=True)
            )
        elif order == 'desc':
            df[f'{column_name}_partitioned_percentile'] = (
                df.groupby(partition_column, group_keys=False)[column_name]
                  .apply(lambda x: pd.qcut(-x, q=100, labels=False, duplicates='drop') + 1)
                  .reset_index(level=0, drop=True)
            )
    
    return df



In [93]:
def create_percentile_look_up_csv(df,column_name_value):
    column_name_percentile = f'{column_name_value}_percentile'
    percentile_look_up = df[[column_name_value, column_name_percentile]].drop_duplicates()

    percentile_look_up.to_csv(f'data/profile_look_up/{snapshot_dt}_{column_name_value}.csv', index=False)
    return


def create_proportion_look_up_csv(df,column_name_value, order):
    column_name_prop = f'{column_name_value}_prop'
    if order == 'asc':
        order = True
    else:
        order = False
    proportion_look_up = df[[column_name_value, column_name_prop]].drop_duplicates()

    proportion_look_up.to_csv(f'data/profile_look_up/{snapshot_dt}_{column_name_value}.csv', index=False)
    proportion_look_up['rank_descending'] = proportion_look_up['player_region_iso_code_long_prop'].rank(method='min', ascending={order})
    return


In [67]:
df,snapshot_dt = load_most_recent_file()

# Dictionary with fill values for each column
fill_values = {
    'career_break_history': 0,   
    'kit': 'No Kit',
    'kit': 'No Kit',
    'kit_shirt_type': 'No Kit Shirt',
    'kit_shirt_logo': 'No Kit Logo',
    'kit_socks_type': 'No Kit Socks',
}

# Fill NaN values in each column with specified values
df = df.fillna(value=fill_values)

In [68]:
# Convert cateogorical to proportion share
columns_to_prop = [
    'player_region_iso_code_long',
    'name_change_blocked',
    'kit',
    'kit_shirt_type',
    'kit_shirt_logo',
    'kit_socks_type'
    ]

for column_name in columns_to_prop:
    df=add_proportion_column(df=df, column_name=column_name)

In [69]:
# Percentile
columns_to_percentile = {
    'classic_leagues_competed_in' : 'desc',
    'h2h_leagues_competed_in' : 'desc',
    'last_deadline_bank' : 'desc',
    'last_deadline_value' : 'desc',
    'last_deadline_total_transfers' : 'desc',
    'summary_overall_points' : 'desc',
    'summary_overall_rank' : 'desc',
    'leagues_admin' : 'desc',
    'min_rank_history' : 'asc',
    'max_total_points_history' : 'desc',
    'earliest_season_year_history' : 'asc',
    'career_break_history' : 'desc',
    'seasons_played_in' : 'desc',
}

# Apply percentiles based on the specified order
df = calculate_percentiles(df=df, columns_to_percentile=columns_to_percentile)

columns_to_percentile_partitioned = [
    {'column_name' : 'stdev_rank_history',
     'partition_column' : 'seasons_played_in',
     'order' : 'desc'
     }
]


df=calculate_percentiles_partitioned(df=df, columns_to_percentile_partitioned=columns_to_percentile_partitioned)

In [76]:
columns_to_percentile = {
    'classic_leagues_competed_in' : 'desc',
    'h2h_leagues_competed_in' : 'desc',
    'last_deadline_bank' : 'desc',
    'last_deadline_value' : 'desc',
    'last_deadline_total_transfers' : 'desc',
    'summary_overall_points' : 'desc',
    'summary_overall_rank' : 'desc',
    'leagues_admin' : 'desc',
    'min_rank_history' : 'asc',
    'max_total_points_history' : 'desc',
    'earliest_season_year_history' : 'asc',
    'career_break_history' : 'desc',
    'seasons_played_in' : 'desc',
}

# Apply percentiles based on the specified order
df = calculate_percentiles(df=df, columns_to_percentile=columns_to_percentile)

columns_to_percentile_partitioned = [
    {'column_name' : 'stdev_rank_history',
     'partition_column' : 'seasons_played_in',
     'order' : 'desc'
     }
]

In [104]:
percentile_cols = ['classic_leagues_competed_in', 'h2h_leagues_competed_in', 'last_deadline_bank', 'last_deadline_value', 'last_deadline_total_transfers', 'summary_overall_points', 'summary_overall_rank', 'leagues_admin', 'min_rank_history', 'max_total_points_history', 'earliest_season_year_history', 'career_break_history', 'seasons_played_in']

In [105]:
for column in percentile_cols:
    create_percentile_look_up_csv(df=df,column_name_value=column)

# NEED TO DO _partitioned_percentile